Seleccionamos el piso del cual queremos mostrar la información en pantalla, y devolverá un fichero con la información del registro seleccionado.

In [1]:
# Carga de las librerías, funciones, y ficheros necesarios.

import numpy as np 
import pandas as pd
import openpyxl

import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split

# Conecta con google drive
from google.colab import drive
drive.mount('/content/drive')

# Carga del dataset de pisos de airbnb Madrid filtrado (13258 reg) sólo por la ciudad Madrid.
airbnbMadrid = pd.read_csv('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/airbnblistingsMadridv1.csv', sep=';', decimal='.')

# Carga de los dataset de los cadenas de supermercados de Madrid.
superMercadona = pd.read_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/SupermercadoMercadona.xlsx')
superCarrefour = pd.read_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/SupermercadoCarrefour.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se introduce por pantalla el numero de piso simulando que es la app de airbnb recopilando la informacion para mostrar al cliente

In [0]:
# Definimos una función para comprobar que el usuario introduce correctamente el ID del piso
def comprueba_int(): 
    while True: 
        try: 
            numEnt = int(input( "Introduzca ID del piso : ")) #Solicita ID piso
            return numEnt 
        except ValueError: 
            print('\nPor favor, introduzca un numero mayor que cero sin decimales\n')
            
# Definimos una función para comprobar que el número introducido sea mayor de 0 : 
def comprueba_mayor_cero(ID): 
    while True: 
        if ID > 0 : 
            return ID
        else:
            print("\nPor favor, introduzca un numero mayor que cero sin decimales\n") 
            ID = comprueba_int()

In [0]:
# Solicita el usuario introduzca el número de identificación del piso. Por ejemplo, el 15332216

IDPiso = 0        # Inicializa el campo a introducir, que deberá ser entero y mayor de cero

#IDPiso = comprueba_mayor_cero(IDPiso)
#print("\nEl ID introducido es :",IDPiso)

IDPiso = 15332216

#airbnbMadridFastFood['ID'][IDPiso]  15332216

In [0]:
import math 

# Función que calcula la distancia geográfica entre dos puntos en metros, pasándole la latitud y la longitud.
def distGeo(latitud1, longitud1, latitud2, longitud2): # posiciones de la vivienda, y los restaurantes
    rad=math.pi/180
    dlatitud=latitud2-latitud1
    dlongitud=longitud2-longitud1
    R=6372.795477598 # Radio de la Tierra
    # La fórmula tiene en cuenta la curvatura del planeta (aunque se podría calcular sin tenerla en cuenta, dado
    # que estamos en la misma ciudad).
    a=(math.sin(rad*dlatitud/2))**2 + math.cos(rad*latitud1)*math.cos(rad*latitud2)*(math.sin(rad*dlongitud/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

Busca el Supermercado Mercadona mas cercano al piso seleccionado

In [0]:
# Mercadona
# Para el piso introducido, calculamos la distancia a todos los Mercadona de Madrid
distanciaMenor = 100  # EL supermercado a menor distancia. Se inicializa a un valor aleatorio. 

# Para el piso introducido, calculamos la distancia a todos los Mercadona de Madrid
latPiso = airbnbMadrid[airbnbMadrid['ID'] == IDPiso]['Latitude']  # latitud del piso 
lonPiso = airbnbMadrid[airbnbMadrid['ID'] == IDPiso]['Longitude'] # longitud del piso

# Para cada Mercadona de Madrid : 
for j in range(len(superMercadona)):
  # Calcula la distancia entre el IDpiso y todos los Mercadona
  distancia=abs(distGeo(latPiso,lonPiso,superMercadona['Latit'][j],superMercadona['Longit'][j]))
  if distancia < distanciaMenor: # si la última distancia calculada es menor que la distancia más pequeña tenemos
     distanciaMenor = distancia  # sustituir la distancia más pequeña que tenemos por la distancia calculada
     IDMenor = superMercadona['ID'][j] # Almacenamos el ID del restaurante
     NombreMenor = superMercadona['Lugar'][j]
     DireccionMenor = superMercadona['Direccion'][j]
     LatitMenor = superMercadona['Latit'][j]
     LongitMenor = superMercadona['Longit'][j]    

In [0]:
# Preparamos los datos
data=[[IDMenor],[NombreMenor],[DireccionMenor],[round(LatitMenor,6)],[round(LongitMenor,6)],[round(distanciaMenor,3)]]
datay=np.transpose(data)

In [0]:
# Creamos y rellenamos el dataframe de coordenadas para el Tableau con los datos del más cercano al piso
# ID : Codigo del establecimiento
# Nombre : nombre del establecimiento
# Direccion
# Latit y Longit : coordenas geográficas
# Distancia : distancia desde el piso seleccionado
dataID = pd.DataFrame(datay, columns=('ID','Nombre','Direccion','Latit','Longit','Distancia'))

Busca el Supermercado Carrefour mas cercano al piso seleccionado

In [0]:
# Carrefour
# Para el piso introducido, calculamos la distancia a todos los Carrefour de Madrid
distanciaMenor = 100  # EL supermercado a menor distancia. Se inicializa a un valor aleatorio. 

# Para cada Carrefour de Madrid : 
for j in range(len(superCarrefour)):
  # Calcula la distancia entre el IDpiso y todos los Carrefour
  distancia=abs(distGeo(latPiso,lonPiso,superCarrefour['Latit'][j],superCarrefour['Longit'][j]))
  if distancia < distanciaMenor: # si la última distancia calculada es menor que la distancia más pequeña tenemos
     distanciaMenor = distancia  # sustituir la distancia más pequeña que tenemos por la distancia calculada
     IDMenor = superCarrefour['ID'][j] # Almacenamos el ID del restaurante
     NombreMenor = superCarrefour['Lugar'][j]
     DireccionMenor = superCarrefour['Direccion'][j]
     LatitMenor = superCarrefour['Latit'][j]
     LongitMenor = superCarrefour['Longit'][j]    

In [0]:
# Añadimos al dataset de salida los datos de Carrefour
data=[IDMenor,NombreMenor,DireccionMenor,round(LatitMenor,6),round(LongitMenor,6),round(distanciaMenor,3)]
datay=np.transpose(data)
dataID.loc[2] = datay

In [0]:
# Por ultimo, añadimos los datos del mismo piso
airbnbMadridID = airbnbMadrid.loc[airbnbMadrid.loc[:, 'ID'] == IDPiso]
airLatitPiso=airbnbMadridID['Latitude'].values
airLongitPiso=airbnbMadridID['Longitude'].values

data=[IDPiso,'Piso','Ud.esta aqui',round(airLatitPiso[0],6),round(airLongitPiso[0],6),0]
datay=np.transpose(data)
dataID.loc[3] = datay

In [11]:
dataID

,ID,Nombre,Direccion,Latit,Longit,Distancia
0,Mer37,Mercadona,"Paseo Gral. Martínez Campos, 39",40.435204,-3.692019,0.916
2,Carr28,Carrefour,Calle Hermosilla 28,40.426336,-3.684699,0.267
3,15332216,Piso,Ud.esta aqui,40.428671,-3.685423,0


Saca por fichero los datos reunidos

In [0]:
# Guarda en fichero de salida los datos recopilados.
dataID.to_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/airbnbMadridSupermercados.xlsx')